In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import requests
import pandas as pd

# Replace with your actual API URL and JWT token
API_URL = 'http://localhost:5000/api/courses'
JWT_TOKEN = 'YOUR_JWT_TOKEN'

###########################
# Data Manipulation / Model
###########################

def fetch_data_from_api():
    headers = {'Authorization': f'Bearer {JWT_TOKEN}'}
    response = requests.get(API_URL, headers=headers)
    if response.status_code == 200:
        return pd.DataFrame.from_records(response.json())
    else:
        print("Failed to fetch data from API")
        return pd.DataFrame()

# Fetch the initial data from the API
df = fetch_data_from_api()
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Animal Shelter Dashboard'))),
    html.Center(html.B(html.H2('Chad Nadeau'))),
    html.A(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': 350, 'width': 350, 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'},
            alt="Loading..."
        ),
        href='https://snhu.edu'
    ),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water', 'value': 'Water'},
            {'label': 'Mountain', 'value': 'Mountain'},
            {'label': 'Disaster', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value=None,
        labelStyle={'display': 'inline-block'}
    ),
    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),
    html.Hr(),
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type is None:
        return df.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    # Construct filter query based on user selection
    filter_query = build_filter_query(filter_type)

    headers = {'Authorization': f'Bearer {JWT_TOKEN}', 'Filter': filter_query}
    filtered_response = requests.get(API_URL, headers=headers)

    if filtered_response.status_code == 200:
        filtered_df = pd.DataFrame.from_records(filtered_response.json())
        if '_id' in filtered_df.columns:
            filtered_df.drop(columns=['_id'], inplace=True)

        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns]
        data = filtered_df.to_dict('records')
        return data, columns

    return [], []

def build_filter_query(filter_type):
    if filter_type == 'Water':
        return {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesa Bay Retr Mix', 'Newfoundland Mix',
                              'Newfoundland/Labrador Retriever', 'Newfoundland/Australian Cattle Dog',
                              'Newfoundland/Great Pyrenees']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 25, '$lte': 157}
        }
    elif filter_type == 'Mountain':
        return {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 25, '$lte': 157}
        }
    elif filter_type == 'Disaster':
        return {
            'breed': {'$in': ['Doberman Pinsch', 'Doberman Pinsch Mix', 'German Shepherd',
                              'Golden Retriever', 'Golden Retriever Mix', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 19, '$lte': 301}
        }
    return {}

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data"),
     Input('filter-type', 'value')]
)
def update_graphs(data, filter_type):
    if filter_type is None or filter_type == 'Reset':
        return []

    dff = pd.DataFrame.from_records(data)

    if not dff.empty:
        names = dff['breed'].value_counts().keys().tolist()
        values = dff['breed'].value_counts().tolist()
        return [
            dcc.Graph(
                figure=px.pie(
                    names=names,
                    values=values,
                    color_discrete_sequence=px.colors.sequential.RdBu,
                    width=800,
                    height=500
                )
            )
        ]
    else:
        return []

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '800px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

print('Connection Successful!')
app.run_server(debug=True)

Connection Successful!
Dash app running on http://127.0.0.1:26774/
